# 📊 Analyse Exploratoire - Performance Retail Géolocalisée

## Objectifs
- Analyser la performance des magasins (CA, clients, panier moyen)
- Identifier les corrélations avec les variables géodémographiques
- Visualiser la répartition géospatiale des performances
- Détecter les facteurs clés de succès

In [ ]:
# Import des librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium import plugins
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

## 1. 📥 Chargement et exploration des données

In [ ]:
# Chargement des données
df_magasins = pd.read_csv('../data/magasins_performance.csv')
df_concurrents = pd.read_csv('../data/sites_concurrents.csv')

print(f"📊 Dataset magasins: {df_magasins.shape[0]} lignes, {df_magasins.shape[1]} colonnes")
print(f"🏪 Dataset concurrents: {df_concurrents.shape[0]} lignes, {df_concurrents.shape[1]} colonnes")

# Aperçu des données
display(df_magasins.head())

In [ ]:
# Informations générales
print("🔍 Informations générales:")
print(df_magasins.info())

print("\n📈 Statistiques descriptives:")
display(df_magasins.describe())

## 2. 📊 Analyses statistiques des performances

In [ ]:
# KPIs clés
ca_total = df_magasins['ca_annuel'].sum()
ca_moyen = df_magasins['ca_annuel'].mean()
panier_moyen_reseau = df_magasins['panier_moyen'].mean()
nb_clients_total = df_magasins['nb_clients_mois'].sum() * 12

print(f"💰 CA total réseau: {ca_total:,.0f}€")
print(f"📍 CA moyen par magasin: {ca_moyen:,.0f}€")
print(f"🛒 Panier moyen réseau: {panier_moyen_reseau:.2f}€")
print(f"👥 Clients annuels estimés: {nb_clients_total:,.0f}")

In [ ]:
# Visualisation des distributions de performance
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# CA annuel
axes[0,0].hist(df_magasins['ca_annuel']/1000, bins=15, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribution du CA annuel')
axes[0,0].set_xlabel('CA annuel (k€)')
axes[0,0].set_ylabel('Nombre de magasins')

# Panier moyen
axes[0,1].hist(df_magasins['panier_moyen'], bins=15, alpha=0.7, color='lightgreen')
axes[0,1].set_title('Distribution du panier moyen')
axes[0,1].set_xlabel('Panier moyen (€)')
axes[0,1].set_ylabel('Nombre de magasins')

# Nombre de clients
axes[1,0].hist(df_magasins['nb_clients_mois'], bins=15, alpha=0.7, color='coral')
axes[1,0].set_title('Distribution du nombre de clients/mois')
axes[1,0].set_xlabel('Clients par mois')
axes[1,0].set_ylabel('Nombre de magasins')

# Surface de vente
axes[1,1].hist(df_magasins['surface_vente'], bins=15, alpha=0.7, color='gold')
axes[1,1].set_title('Distribution de la surface de vente')
axes[1,1].set_xlabel('Surface (m²)')
axes[1,1].set_ylabel('Nombre de magasins')

plt.tight_layout()
plt.show()

## 3. 🗺️ Analyse géospatiale

In [ ]:
# Carte interactive des magasins avec performance
# Centre de la carte sur la France
m = folium.Map(location=[46.6034, 1.8883], zoom_start=6)

# Normalisation du CA pour la taille des marqueurs
ca_min, ca_max = df_magasins['ca_annuel'].min(), df_magasins['ca_annuel'].max()

# Couleurs par enseigne
colors = {'SuperFrais': 'blue', 'MarchéPlus': 'green', 'BioNature': 'orange', 
         'CityMarket': 'red', 'FamilyShop': 'purple'}

for idx, row in df_magasins.iterrows():
    # Taille proportionnelle au CA
    size = 5 + 15 * (row['ca_annuel'] - ca_min) / (ca_max - ca_min)
    
    popup_text = f"""
    <b>{row['enseigne']} - {row['format']}</b><br>
    📍 {row['ville']}<br>
    💰 CA: {row['ca_annuel']:,.0f}€<br>
    🛒 Panier: {row['panier_moyen']:.2f}€<br>
    👥 Clients/mois: {row['nb_clients_mois']:,}<br>
    🏪 Surface: {row['surface_vente']}m²
    """
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=size,
        popup=popup_text,
        color=colors.get(row['enseigne'], 'gray'),
        fill=True,
        weight=2,
        opacity=0.8
    ).add_to(m)

# Affichage de la carte
m

In [ ]:
# Analyse par ville
perf_ville = df_magasins.groupby('ville').agg({
    'ca_annuel': ['sum', 'mean', 'count'],
    'panier_moyen': 'mean',
    'nb_clients_mois': 'sum'
}).round(2)

perf_ville.columns = ['CA_total', 'CA_moyen', 'Nb_magasins', 'Panier_moyen', 'Clients_total']
perf_ville = perf_ville.sort_values('CA_total', ascending=False)

print("🏙️ Performance par ville:")
display(perf_ville.head(10))

## 4. 🔍 Analyses de corrélations

In [ ]:
# Variables numériques pour l'analyse de corrélation
variables_numeriques = [
    'ca_annuel', 'panier_moyen', 'nb_clients_mois', 'surface_vente',
    'population_zone_1km', 'densite_hab_km2', 'revenu_median_zone',
    'concurrents_500m', 'concurrents_1km', 'parking_places',
    'distance_centre_ville', 'transport_score'
]

# Matrice de corrélation
corr_matrix = df_magasins[variables_numeriques].corr()

# Heatmap des corrélations
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Matrice de corrélation - Variables géodémographiques vs Performance')
plt.tight_layout()
plt.show()

In [ ]:
# Corrélations avec le CA (tri décroissant)
corr_ca = corr_matrix['ca_annuel'].drop('ca_annuel').sort_values(key=abs, ascending=False)

print("🎯 Variables les plus corrélées au CA annuel:")
for var, corr in corr_ca.items():
    direction = "📈" if corr > 0 else "📉"
    print(f"{direction} {var}: {corr:.3f}")

In [ ]:
# Scatter plots des variables les plus corrélées
top_vars = corr_ca.head(4).index.tolist()

fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.ravel()

for i, var in enumerate(top_vars):
    axes[i].scatter(df_magasins[var], df_magasins['ca_annuel']/1000, alpha=0.6)
    axes[i].set_xlabel(var)
    axes[i].set_ylabel('CA annuel (k€)')
    axes[i].set_title(f'CA vs {var}\n(r={corr_ca[var]:.3f})')
    
    # Ligne de tendance
    z = np.polyfit(df_magasins[var], df_magasins['ca_annuel'], 1)
    p = np.poly1d(z)
    axes[i].plot(df_magasins[var], p(df_magasins[var])/1000, "r--", alpha=0.8)

plt.tight_layout()
plt.show()

## 5. 📊 Performance par enseigne et format

In [ ]:
# Performance par enseigne
perf_enseigne = df_magasins.groupby('enseigne').agg({
    'ca_annuel': ['mean', 'sum', 'count'],
    'panier_moyen': 'mean',
    'nb_clients_mois': 'mean'
}).round(0)

perf_enseigne.columns = ['CA_moyen', 'CA_total', 'Nb_magasins', 'Panier_moyen', 'Clients_mois']

print("🏪 Performance par enseigne:")
display(perf_enseigne.sort_values('CA_moyen', ascending=False))

In [ ]:
# Graphiques de performance par enseigne
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# CA moyen par enseigne
perf_enseigne['CA_moyen'].plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('CA moyen par enseigne')
axes[0].set_ylabel('CA annuel (€)')
axes[0].tick_params(axis='x', rotation=45)

# Panier moyen par enseigne
perf_enseigne['Panier_moyen'].plot(kind='bar', ax=axes[1], color='lightgreen')
axes[1].set_title('Panier moyen par enseigne')
axes[1].set_ylabel('Panier moyen (€)')
axes[1].tick_params(axis='x', rotation=45)

# Nombre de clients par enseigne
perf_enseigne['Clients_mois'].plot(kind='bar', ax=axes[2], color='coral')
axes[2].set_title('Clients/mois par enseigne')
axes[2].set_ylabel('Clients par mois')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. 🎯 Facteurs clés de succès

In [ ]:
# Segmentation des magasins par performance
df_magasins['segment_ca'] = pd.cut(df_magasins['ca_annuel'], 
                                  bins=3, 
                                  labels=['Faible', 'Moyen', 'Fort'])

# Analyse des segments
analyse_segments = df_magasins.groupby('segment_ca').agg({
    'ca_annuel': 'mean',
    'population_zone_1km': 'mean',
    'revenu_median_zone': 'mean',
    'parking_places': 'mean',
    'concurrents_500m': 'mean',
    'transport_score': 'mean',
    'surface_vente': 'mean'
}).round(0)

print("🎯 Profil des segments de performance:")
display(analyse_segments)

In [ ]:
# Recommandations basées sur l'analyse
print("📋 SYNTHÈSE ET RECOMMANDATIONS")
print("="*50)

# Top corrélations positives
top_positive = corr_ca[corr_ca > 0].head(3)
print("\n✅ FACTEURS DE SUCCÈS:")
for var, corr in top_positive.items():
    print(f"   • {var}: +{corr:.3f}")

# Top corrélations négatives
top_negative = corr_ca[corr_ca < 0].tail(3)
print("\n⚠️ FACTEURS DE RISQUE:")
for var, corr in top_negative.items():
    print(f"   • {var}: {corr:.3f}")

# Recommandations d'implantation
print("\n🎯 CRITÈRES D'IMPLANTATION OPTIMAUX:")
best_performers = df_magasins[df_magasins['segment_ca'] == 'Fort']
print(f"   • Population zone 1km: {best_performers['population_zone_1km'].mean():.0f} habitants")
print(f"   • Revenu médian zone: {best_performers['revenu_median_zone'].mean():.0f}€")
print(f"   • Places de parking: {best_performers['parking_places'].mean():.0f}")
print(f"   • Score transport: {best_performers['transport_score'].mean():.1f}/10")
print(f"   • Concurrents 500m max: {best_performers['concurrents_500m'].mean():.1f}")